In [1]:
using JuMP
import Pkg
Pkg.add("Gurobi")
import Gurobi
using XLSX, DataFrames
using Random
import Test

#instead of CSV, well use XLSX to read multiple sheets


   Resolving package versions...
  No Changes to `C:\Users\Joe\OneDrive\Documents\GitHub\LPGenie\Project.toml`
  No Changes to `C:\Users\Joe\OneDrive\Documents\GitHub\LPGenie\Manifest.toml`


In [2]:
upload =  XLSX.readxlsx("UploadTemplate.xlsx")
# Instructions, and sample sheets will NOT be read
#put this file in the same directory as the trialjoe.ipynb  

XLSXFile("UploadTemplate.xlsx") containing 9 Worksheets
            sheetname size          range        
-------------------------------------------------
         INSTRUCTIONS 29x2          A1:B29       
               demand 500x17        A1:Q500      
        workerproduct 20x63         A1:BK20      
    initial_inventory 7x49          A1:AW7       
       initial_worker 7x28          A1:AB7       
        sample_demand 500x17        A1:Q500      
 sample_workerproduct 20x63         A1:BK20      
sample_initial_inven… 7x49          A1:AW7       
sample_initial_worke… 7x28          A1:AB7       


In [3]:
demand_df = DataFrame(XLSX.readtable("UploadTemplate.xlsx","demand")) #Note to self: UploadTemplate.xlsx name in app.jl would be joinpath(FILE_PATH,filename) variable. 

Row,Date,Product 1,Product 2,Product 3,Product 4
,Any,Any,Any,Any,Any
1,2024-01-01,4780,2612,1428,860
2,2024-02-01,4736,5232,1820,828
3,2024-03-01,4780,2332,3176,756
4,2024-04-01,7548,3380,2488,832
5,2024-05-01,6312,4240,2368,660
6,2024-06-01,3324,2252,2916,948
7,2024-07-01,5420,384,1616,616
8,2024-08-01,6064,2040,2496,536
9,2024-09-01,7604,4776,1908,696


In [4]:
workerproduct_df = DataFrame(XLSX.readtable("UploadTemplate.xlsx","workerproduct"))

Row,Worker Type,Product 1,Product 2,Product 3,Product 4
,Any,Any,Any,Any,Any
1,workertype_1,80,86,83,90
2,workertype_2,136,140,145,150
3,workertype_3,100,102,90,120


In [5]:
initial_inventory_df = DataFrame(XLSX.readtable("UploadTemplate.xlsx","initial_inventory"))

Row,Product 1,Product 2,Product 3,Product 4
,Any,Any,Any,Any
1,150,120,30,0


In [6]:
initial_worker_df = DataFrame(XLSX.readtable("UploadTemplate.xlsx","initial_worker"))

Row,workertype 1,workertype 2,workertype 3
,Any,Any,Any
1,87,89,50


In [7]:
function read_forecast(demand_df)
    num_products_p = ncol(demand_df) - 1
    println(num_products_p)  #to not include the time columnindex
    time_horizon_T = nrow(demand_df)
    println(time_horizon_T)
    date_list = demand_df[:,1]
    date_difference = date_list[2] - date_list[1]
    initial_date = date_list[1] - date_difference
    date_list = [initial_date; date_list]
    date_list = [string(date) for date in date_list]
    println(date_list)
    product_names = names(demand_df)[2:end]
    println(product_names)
    demand_data = Matrix(demand_df[:, 2:end])
    demand_D = transpose(demand_data)
    println(demand_D)

    return num_products_p,time_horizon_T,product_names,demand_D,date_list
end    

num_products_p,time_horizon_T,product_names,demand_D,date_list = read_forecast(demand_df)

4
24
["2023-12-01", "2024-01-01", "2024-02-01", "2024-03-01", "2024-04-01", "2024-05-01", "2024-06-01", "2024-07-01", "2024-08-01", "2024-09-01", "2024-10-01", "2024-11-01", "2024-12-01", "2025-01-01", "2025-02-01", "2025-03-01", "2025-04-01", "2025-05-01", "2025-06-01", "2025-07-01", "2025-08-01", "2025-09-01", "2025-10-01", "2025-11-01", "2025-12-01"]
["Product 1", "Product 2", "Product 3", "Product 4"]
Any[4780 4736 4780 7548 6312 3324 5420 6064 7604 5684 4436 7068 6480 6776 2648 2960 6468 3312 7868 6520 5100 4192 4432 2132; 2612 5232 2332 3380 4240 2252 384 2040 4776 3068 4484 3140 1332 1284 3544 0 2472 284 2968 3524 908 3912 5120 4044; 1428 1820 3176 2488 2368 2916 1616 2496 1908 1548 2592 1460 1532 2456 2300 3596 2888 2696 3724 2060 2020 3008 2080 1656; 860 828 756 832 660 948 616 536 696 764 256 984 880 708 692 816 780 784 672 544 708 768 904 636]


(4, 24, ["Product 1", "Product 2", "Product 3", "Product 4"], Any[4780 4736 … 4432 2132; 2612 5232 … 5120 4044; 1428 1820 … 2080 1656; 860 828 … 904 636], ["2023-12-01", "2024-01-01", "2024-02-01", "2024-03-01", "2024-04-01", "2024-05-01", "2024-06-01", "2024-07-01", "2024-08-01", "2024-09-01"  …  "2025-03-01", "2025-04-01", "2025-05-01", "2025-06-01", "2025-07-01", "2025-08-01", "2025-09-01", "2025-10-01", "2025-11-01", "2025-12-01"])

In [8]:

#Using Angela's code, return workertype_L back to workertype_W, num_workertype_W to num_workertype_w



function read_workertype(workerproduct_df)
    num_workertype_w=nrow(workerproduct_df)
    println(num_workertype_w)
    worker_names= workerproduct_df[:,1]
    println(worker_names) 
    productionrate_K = Matrix(workerproduct_df[1:end,2:end])
    println(productionrate_K)
    return num_workertype_w, worker_names, productionrate_K
end

num_workertype_w, worker_names, productionrate_K = read_workertype(workerproduct_df)

3
Any["workertype_1", "workertype_2", "workertype_3"]
Any[80 86 83 90; 136 140 145 150; 100 102 90 120]


(3, Any["workertype_1", "workertype_2", "workertype_3"], Any[80 86 83 90; 136 140 145 150; 100 102 90 120])

In [9]:
initial_inventory_I0 = Matrix(initial_inventory_df[!,1:end])
println(initial_inventory_I0)
println(initial_inventory_I0[2])

Any[150 120 30 0]
120


In [10]:
initial_worker_W0 = Matrix(initial_worker_df[!,1:end])
println(initial_worker_W0)
println(initial_worker_W0[2])
println(num_workertype_w)

Any[87 89 50]
89
3


In [11]:
# function read_forecast(forecast)
#     num_products_p = ncol(forecast) - 1  #to not include the time columnindex
#     time_horizon_T = nrow(forecast)
#     product_names = names(forecast)[2:end]

#     #Initialise dates in correct format
#     date_list = []
#     date_input_format = DateFormat("dd/mm/yyyy")
#     date_output_format = DateFormat("yyyy-mm-dd")
#     for row in 1:time_horizon_T
#         date_obj = Date(forecast[row,1],date_input_format)
#         formatted_date_obj = Dates.format(date_obj,date_output_format)
#         push!(date_list,formatted_date_obj)
#     end

#     return num_products_p,time_horizon_T,product_names,demand_D,workdays_n,productivity_K,date_list
# end

In [12]:
println(num_workertype_w)
println(num_products_p)
println(time_horizon_T)

3
4
24


In [13]:
# Initialise Costs
cost_hiring_cH = 400    # Hiring cost per worker
cost_firing_cF = 700    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 233.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 999999  #Cost of Backlogging per overtime unit 
cost_idle_cU= 250 #Cost of Idle per worker
cost_subcontract_cS= 350 #cost of Subcontract 

350

In [14]:
#Attempt
using JuMP
using Gurobi

# Define the model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, workerlevel_W[1:num_workertype_w,1:num_products_p, 0:time_horizon_T] >= 0, Int) # Workers
@variable(model, hired_H[1:num_workertype_w,0:time_horizon_T] >= 0, Int)       # Hired workers
@variable(model, fired_F[1:num_workertype_w,0:time_horizon_T] >= 0, Int)       # Fired workers
@variable(model, inventory_I[1:num_products_p, 0:time_horizon_T] >= 0, Int)     # Inventory
@variable(model, production_P[1:num_products_p, 0:time_horizon_T] >= 0, Int)    # Production
@variable(model, overtime_O[1:num_products_p, 0:time_horizon_T] >= 0, Int)      # Overtime
@variable(model, backlogging_B[1:num_products_p, 0:time_horizon_T] >= 0, Int)   # Backlogging
@variable(model, idle_U[1:num_products_p, 0:time_horizon_T] >= 0, Int)          # Idle
@variable(model, subcontract_S[1:num_products_p, 0:time_horizon_T] >= 0, Int) #Subcontract
@variable(model, regularproduction_R[1:num_products_p, 0:time_horizon_T] >= 0, Int) #Regular Production
@variable(model, aggregated_workerlevel_Wwt[1:num_workertype_w, 0:time_horizon_T])    #Worker Level but 2D

# Objective
@objective(model, Min,
    sum(cost_hiring_cH * hired_H[w, t] + cost_firing_cF * fired_F[w, t] for w in 1:num_workertype_w, t in 0:time_horizon_T) +
    sum(cost_inventory_cI * inventory_I[p, t] + cost_labour_cR * production_P[p, t] +
        cost_overtime_cO * overtime_O[p, t] + cost_idle_cU * idle_U[p, t] +
        cost_subcontract_cS * subcontract_S[p, t] + cost_backlogging_cB * backlogging_B[p, t]
        for p in 1:num_products_p, t in 0:time_horizon_T)
)

# Constraints
for w in 1:num_workertype_w
    for t in 1:time_horizon_T
        @constraint(model, aggregated_workerlevel_Wwt[w, t] == sum(workerlevel_W[w, p, t] for p in 1:num_products_p))
    end
end

#set initial values of workertype variables for 0 for time period 0

for w in 1:num_workertype_w
    @constraint(model, aggregated_workerlevel_Wwt[w, 0] == initial_worker_W0[w])
    @constraint(model, hired_H[w,0] == 0)
    @constraint(model, fired_F[w,0] == 0)
end

#set initial values of production variables to 0 for time period 0

for p in 1:num_products_p
    @constraint(model, inventory_I[p,0] == initial_inventory_I0[p])
    @constraint(model, backlogging_B[p,0] == 0 )
    @constraint(model, overtime_O[p,0] == 0)
    @constraint(model, subcontract_S[p,0] == 0)
    @constraint(model, production_P[p,0] == 0)
    @constraint(model, regularproduction_R[p,0] == 0)
    @constraint(model, idle_U[p,0] == 0)
end     

for w in 1:num_workertype_w
    for t in 1: time_horizon_T
        @constraint(model, aggregated_workerlevel_Wwt[w,t] == aggregated_workerlevel_Wwt[w, t-1] + hired_H[w,t] - fired_F[w,t])   
    end    
end

for w in 1:num_workertype_w
    for p in 1:num_products_p
        for t in 1:time_horizon_T
            @constraint(model, regularproduction_R[p,t] <= productionrate_K[w,p]*workerlevel_W[w,p,t]) 
        end
    end
end

# Worker level constraints for subsequent periods
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, production_P[p,t] == regularproduction_R[p,t] + overtime_O[p,t] - idle_U[p,t])
        @constraint(model, overtime_O[p,t] <= 0.2*regularproduction_R[p,t])
        #@constraint(model, hired_H[p,t] + fired[w,t] <= 0.1*sum(workerlevel_W[w,p,t]) for w in num_workerlevel_W)
    end
end

# Inventory and backlogging constraints for subsequent periods
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, inventory_I[p,t] - backlogging_B[p,t] == inventory_I[p,t-1] + backlogging_B[p,t-1] + production_P[p,t] + subcontract_S[p,t] - demand_D[p,t])    
    end
end

# Solve the model
optimize!(model)

println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_W)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
println("Idle: ", round.(Int,value.(idle_U)))
println("Subcontract: ", round.(Int,value.(subcontract_S)))
println("Regular Production ",round.(Int,value.(regularproduction_R)))

print(model)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Optimize a model with 757 rows, 1225 columns and 2413 nonzeros
Model fingerprint: 0xae41e3c4
Variable types: 75 continuous, 1150 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+02]
  Objective range  [9e+00, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 8e+03]
Found heuristic solution: objective 8.558686e+09
Presolve removed 175 rows and 607 columns
Presolve time: 0.00s
Presolved: 582 rows, 618 columns, 1609 

In [23]:
#full optimise function testing

cost_hiring_cH = 5882    # Hiring cost per worker
cost_firing_cF = 896    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 233.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 999999  #Cost of Backlogging per overtime unit 
cost_idle_cU= 250  #Cost of Idle per worker
cost_subcontract_cS= 350  #cost of Subcontract 
productionrate_K = Any[120 137 86 137; 122 140 100 150; 130 125 90 120]

initial_inventory_I0 = Any[150 120 30 0]
initial_worker_W0 = Any[42 43 23]
num_products_p = 4
num_workertype_w = 3
date_list = ["2023-12-01", "2024-01-01", "2024-02-01", "2024-03-01", "2024-04-01", "2024-05-01", "2024-06-01", "2024-07-01", "2024-08-01", "2024-09-01", "2024-10-01", "2024-11-01", "2024-12-01", "2025-01-01", "2025-02-01", "2025-03-01", "2025-04-01", "2025-05-01", "2025-06-01", "2025-07-01", "2025-08-01", "2025-09-01", "2025-10-01", "2025-11-01", "2025-12-01"]
time_horizon_T = 24
demand_D = Any[5860 7500 5896 2964 5188 1464 3880 4192 2580 5328 5208 6032 4344 6728 5084 4264 5840 3716 5352 3032 3040 4596 2460 3916; 2392 4300 2468 6064 2304 1092 3640 132 3684 4956 3612 2760 2784 3468 30 3128 3504 4660 1648 2804 3872 440 5660 1840; 2128 2860 556 2340 3004 1776 2032 2464 1068 2936 1736 1448 2364 2684 2116 3604 2932 3072 1552 1628 3232 2120 2708 2176; 616 812 612 1276 728 900 576 472 652 624 576 576 1180 628 720 780 984 652 1004 664 560 932 824 452]
product_names = ["XC_60 Gent", "XC_60 China", "Kuga", "Mondeo"]
worker_names = Any["Assembler", "Packer", "Tester"]

function optimise(num_products_p,num_workertype_w,time_horizon_T,demand_D,productionrate_K,cost_hiring_cH, cost_firing_cF,cost_inventory_cI,cost_labour_cR,cost_overtime_cO,cost_backlogging_cB,cost_idle_cU,cost_subcontract_cS,product_names,date_list,worker_names, initial_inventory_I0, initial_worker_W0)
    model = Model(Gurobi.Optimizer)

    # Variables
    @variable(model, workerlevel_W[1:num_workertype_w,1:num_products_p, 0:time_horizon_T] >= 0, Int) # Workers
    @variable(model, hired_H[1:num_workertype_w,0:time_horizon_T] >= 0, Int)       # Hired workers
    @variable(model, fired_F[1:num_workertype_w,0:time_horizon_T] >= 0, Int)       # Fired workers
    @variable(model, inventory_I[1:num_products_p, 0:time_horizon_T] >= 0, Int)     # Inventory
    @variable(model, production_P[1:num_products_p, 0:time_horizon_T] >= 0, Int)    # Production
    @variable(model, overtime_O[1:num_products_p, 0:time_horizon_T] >= 0, Int)      # Overtime
    @variable(model, backlogging_B[1:num_products_p, 0:time_horizon_T] >= 0, Int)   # Backlogging
    @variable(model, idle_U[1:num_products_p, 0:time_horizon_T] >= 0, Int)          # Idle
    @variable(model, subcontract_S[1:num_products_p, 0:time_horizon_T] >= 0, Int) #Subcontract
    @variable(model, regularproduction_R[1:num_products_p, 0:time_horizon_T] >= 0, Int) #Regular Production
    @variable(model, aggregated_workerlevel_Wwt[1:num_workertype_w, 0:time_horizon_T])    #Worker Level but 2D

    # Objective
    @objective(model, Min,
        sum(cost_hiring_cH * hired_H[w, t] + cost_firing_cF * fired_F[w, t] for w in 1:num_workertype_w, t in 0:time_horizon_T) +
        sum(cost_inventory_cI * inventory_I[p, t] + cost_labour_cR * production_P[p, t] +
            cost_overtime_cO * overtime_O[p, t] + cost_idle_cU * idle_U[p, t] +
            cost_subcontract_cS * subcontract_S[p, t] + cost_backlogging_cB * backlogging_B[p, t]
            for p in 1:num_products_p, t in 0:time_horizon_T)
    )

    # Constraints
    for w in 1:num_workertype_w
        for t in 1:time_horizon_T
            @constraint(model, aggregated_workerlevel_Wwt[w, t] == sum(workerlevel_W[w, p, t] for p in 1:num_products_p))
        end
    end

    #set initial values of workertype variables for 0 for time period 0

    for w in 1:num_workertype_w
        @constraint(model, aggregated_workerlevel_Wwt[w, 0] == initial_worker_W0[w])
        @constraint(model, hired_H[w,0] == 0)
        @constraint(model, fired_F[w,0] == 0)
    end

    #set initial values of production variables to 0 for time period 0

    for p in 1:num_products_p
        @constraint(model, inventory_I[p,0] == initial_inventory_I0[p])
        @constraint(model, backlogging_B[p,0] == 0 )
        @constraint(model, overtime_O[p,0] == 0)
        @constraint(model, subcontract_S[p,0] == 0)
        @constraint(model, production_P[p,0] == 0)
        @constraint(model, regularproduction_R[p,0] == 0)
        @constraint(model, idle_U[p,0] == 0)
    end     

    for w in 1:num_workertype_w
        for t in 1: time_horizon_T
            @constraint(model, aggregated_workerlevel_Wwt[w,t] == aggregated_workerlevel_Wwt[w, t-1] + hired_H[w,t] - fired_F[w,t])
            #@constraint(model, hired_H[w,t] + fired_F[w,t] <= 0.2*aggregated_workerlevel_Wwt[w,t])   
        end    
    end

    for w in 1:num_workertype_w
        for p in 1:num_products_p
            for t in 1:time_horizon_T
                @constraint(model, regularproduction_R[p,t] <= productionrate_K[w,p]*workerlevel_W[w,p,t]) 
            end
        end
    end

    # Worker level constraints for subsequent periods
    for p in 1:num_products_p
        for t in 1:time_horizon_T
            @constraint(model, production_P[p,t] == regularproduction_R[p,t] + overtime_O[p,t] - idle_U[p,t])
            @constraint(model, overtime_O[p,t] <= 0.2*regularproduction_R[p,t])
        end
    end

    # Inventory and backlogging constraints for subsequent periods
    for p in 1:num_products_p
        for t in 1:time_horizon_T
            @constraint(model, inventory_I[p,t] - backlogging_B[p,t] == inventory_I[p,t-1] - backlogging_B[p,t-1] + production_P[p,t] + subcontract_S[p,t] - demand_D[p,t])    
        end
    end

    # Solve the model
    optimize!(model)

    #update to rounded to int values
    production_P_round = round.(Int,value.(production_P))
    overtime_O_round = round.(Int,value.(overtime_O))
    backlogging_B_round = round.(Int,value.(backlogging_B))
    idle_U_round = round.(Int,value.(idle_U))
    subcontract_S_round = round.(Int,value.(subcontract_S))
    inventory_I_round = round.(Int,value.(inventory_I))
    zero_column_round = round.(Int,zeros(num_products_p,1)) #for demand, because it is 4x24, didnt include the intiial time period
    demand_D_round = hcat(zero_column_round, demand_D)
    fired_F_round = round.(Int,value.(fired_F))
    hired_H_round = round.(Int,value.(hired_H))
    aggregated_workerlevel_Wwt_round = round.(Int,value.(aggregated_workerlevel_Wwt)) #transforms demand into 4x25

    # for debugging purposes
    println("Objective value: ", objective_value(model))
    println("Workers: ", aggregated_workerlevel_Wwt_round)
    println("Hired: ", hired_H_round)
    println("Fired: ", fired_F_round)
    println("Inventory: ", inventory_I_round)
    println("Production: ", production_P_round)
    println("Overtime: ", overtime_O_round)
    println("Backlogging: ", backlogging_B_round)
    println("Idle: ", idle_U_round)
    println("Subcontract: ", subcontract_S_round)

    production_df = DataFrame(
        Product_Name = repeat(product_names, inner = time_horizon_T+1), #to include the initial date (NOT in forecast)
        Date = repeat(date_list, outer = num_products_p),
        Demand = collect(Iterators.flatten(eachrow(demand_D_round))),
        Inventory = collect(Iterators.flatten(eachrow(inventory_I_round))),
        Production = collect(Iterators.flatten(eachrow(production_P_round))),
        Overtime = collect(Iterators.flatten(eachrow(overtime_O_round))),
        Backlogging = collect(Iterators.flatten(eachrow(backlogging_B_round))),
        Idle = collect(Iterators.flatten(eachrow(idle_U_round))),
        Subcontract = collect(Iterators.flatten(eachrow(subcontract_S_round))),
    )

    worker_df = DataFrame(
    Worker_Type = repeat(worker_names, inner = time_horizon_T+1),
    Date = repeat(date_list, outer = num_workertype_w),
    Workers_Fired = collect(Iterators.flatten(eachrow(fired_F_round))),
    Workers_Hired = collect(Iterators.flatten(eachrow(hired_H_round))),
    Worker_Level = collect(Iterators.flatten(eachrow(aggregated_workerlevel_Wwt_round)))
    )

    return objective_value(model), aggregated_workerlevel_Wwt_round, hired_H_round, fired_F_round, inventory_I_round, production_P_round, overtime_O_round, backlogging_B_round, idle_U_round, subcontract_S_round, worker_df, production_df
end  

optimise (generic function with 1 method)

In [24]:
result = optimise(num_products_p,num_workertype_w,time_horizon_T,demand_D,productionrate_K,cost_hiring_cH, cost_firing_cF,cost_inventory_cI,cost_labour_cR,cost_overtime_cO,cost_backlogging_cB,cost_idle_cU,cost_subcontract_cS,product_names,date_list,worker_names)

LoadError: MethodError: no method matching optimise(::Int64, ::Int64, ::Int64, ::Matrix{Any}, ::Matrix{Any}, ::Int64, ::Int64, ::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64, ::Vector{String}, ::Vector{String}, ::Vector{Any})

[0mClosest candidates are:
[0m  optimise(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, [91m::Any[39m, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[23]:23[24m[39m


In [25]:
println(result[1])
println(result[2])
println(result[3])
println(result[4])
println(result[5])
println(result[6])
println(result[7])
println(result[8])
println(result[9])
println(result[10])
println(result[11])
println(result[12])
#println(result[13])

LoadError: UndefVarError: `result` not defined

In [26]:
#now i have 25 periods, i want to cut period 0
value.(inventory_I)


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(4)
    Dimension 2, 0:24
And data, a 4×25 Matrix{Float64}:
 150.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
 120.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  0.0
  30.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  0.0
   0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  0.0

In [ ]:
print(value.(inventory_I))
print(value.(backlogging_B))
#remove time period 0

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(4)
    Dimension 2, 0:24
And data, a 4×25 Matrix{Float64}:
 150.0   -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
 120.0   53.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   2.0  -0.0  -0.0   2.0  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
  30.0  925.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   2.0  -0.0  -0.0   2.0  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
   0.0   62.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   2.0  -0.0  0.02-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(4)
    Dimension 2, 0:24
And data, a 4×25 Matrix{Float64}:
 0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0 

In [ ]:

production_P_round = round.(Int,value.(production_P))
overtime_O_round = round.(Int,value.(overtime_O))
backlogging_B_round = round.(Int,value.(backlogging_B))
idle_U_round = round.(Int,value.(idle_U))
subcontract_S_round = round.(Int,value.(subcontract_S))
inventory_I_round = round.(Int,value.(inventory_I))
zero_column_round = round.(Int,zeros(num_products_p,1)) #for demand, because it is 4x24, didnt include the intiial time period
demand_D_round = hcat(zero_column_round, demand_D) #transforms demand into 4x25

4×25 Matrix{Any}:
 0  5860  7500  5896  2964  5188  1464  …  5352  3032  3040  4596  2460  3916
 0  2392  4300  2468  6064  2304  1092     1648  2804  3872   440  5660  1840
 0  2128  2860   556  2340  3004  1776     1552  1628  3232  2120  2708  2176
 0   616   812   612  1276   728   900     1004   664   560   932   824   452

In [ ]:
production_df = DataFrame(
    Product_Name = repeat(product_names, inner = time_horizon_T+1), #to include the initial date (NOT in forecast)
    Date = repeat(date_list, outer = num_products_p),
    Demand = collect(Iterators.flatten(eachrow(demand_D_round))),
    Inventory = collect(Iterators.flatten(eachrow(inventory_I_round))),
    Production = collect(Iterators.flatten(eachrow(production_P_round))),
    Overtime = collect(Iterators.flatten(eachrow(overtime_O_round))),
    Backlogging = collect(Iterators.flatten(eachrow(backlogging_B_round))),
    Idle = collect(Iterators.flatten(eachrow(idle_U_round))),
    Subcontract = collect(Iterators.flatten(eachrow(subcontract_S_round))),
)

Row,Product_Name,Date,Demand,Inventory,Production,Overtime,Backlogging,Idle,Subcontract
,String,String,Any,Int64,Int64,Int64,Int64,Int64,Int64
1,XC_60 Gent,2023-12-01,0,150,0,0,0,0,0
2,XC_60 Gent,2024-01-01,5860,0,5710,0,0,0,0
3,XC_60 Gent,2024-02-01,7500,0,7500,0,0,0,0
4,XC_60 Gent,2024-03-01,5896,0,5896,0,0,0,0
5,XC_60 Gent,2024-04-01,2964,0,2964,0,0,0,0
6,XC_60 Gent,2024-05-01,5188,0,5188,0,0,0,0
7,XC_60 Gent,2024-06-01,1464,0,1464,0,0,0,0
8,XC_60 Gent,2024-07-01,3880,0,3880,0,0,0,0
9,XC_60 Gent,2024-08-01,4192,0,4192,0,0,0,0


In [ ]:
selected_product = "Kuga" 
start_date = "2023-02-30"
end_date = "2025-03-02"
production_df_copy = copy(production_df)
filter_production_df = filter!(row -> row.Product_Name == selected_product &&  start_date <= row.Date <= end_date, production_df_copy)
demand_plot = filter_production_df.Demand
inventory_plot = filter_production_df.Inventory
production_plot = filter_production_df.Production
overtime_plot = filter_production_df.Overtime
backlogging_plot = filter_production_df.Backlogging
idle_plot = filter_production_df.Idle
subcontract_plot = filter_production_df.Subcontract

println(demand_plot)
println(inventory_plot)
println(production_plot)
println(overtime_plot)
println(backlogging_plot)
println(idle_plot)
println(subcontract_plot)


Any[0, 2128, 2860, 556, 2340, 3004, 1776, 2032, 2464, 1068, 2936, 1736, 1448, 2364, 2684, 2116]
[30, 925, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 3023, 1935, 556, 2340, 3000, 1776, 2032, 2464, 1068, 2936, 1736, 1448, 2364, 2684, 2118]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
#Update to rounded integer values , for clarity
fired_F_round = round.(Int,value.(fired_F))
hired_H_round = round.(Int,value.(hired_H))
println(fired_F_round)
println(hired_H_round)

2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, 0:24
And data, a 3×25 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, 0:24
And data, a 3×25 Matrix{Int64}:
 0  80  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  72  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  96  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0


In [25]:
# Initialize result array with appropriate dimensions
aggregated_workerlevel_Wwt_round = round.(Int,value.(aggregated_workerlevel_Wwt))

println(aggregated_workerlevel_Wwt_round)

2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, 0:24
And data, a 3×25 Matrix{Int64}:
 42  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122  122
 43  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115  115
 23  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119  119


In [26]:
worker_df = DataFrame(
    Worker_Type = repeat(worker_names, inner = time_horizon_T+1),
    Date = repeat(date_list, outer = num_workertype_w),
    Workers_Fired = collect(Iterators.flatten(eachrow(fired_F_round))),
    Workers_Hired = collect(Iterators.flatten(eachrow(hired_H_round))),
    Worker_Level = collect(Iterators.flatten(eachrow(aggregated_workerlevel_Wwt_round)))
)

Row,Worker_Type,Date,Workers_Fired,Workers_Hired,Worker_Level
,Any,String,Int64,Int64,Int64
1,Assembler,2023-12-01,0,0,42
2,Assembler,2024-01-01,0,80,122
3,Assembler,2024-02-01,0,0,122
4,Assembler,2024-03-01,0,0,122
5,Assembler,2024-04-01,0,0,122
6,Assembler,2024-05-01,0,0,122
7,Assembler,2024-06-01,0,0,122
8,Assembler,2024-07-01,0,0,122
9,Assembler,2024-08-01,0,0,122


In [33]:
#for worker graph

selected_worker = "Tester" 
start_date = "2024-01-30"
end_date = "2026-04-02"
worker_df_copy = copy(worker_df)
filter_worker_df = filter!(row -> row.Worker_Type == selected_worker &&  start_date <= row.Date <= end_date, worker_df_copy)
workerlevel_plot = filter_worker_df.Worker_Level
hired_plot = filter_worker_df.Workers_Hired
fired_plot = filter_worker_df.Workers_Fired 
println(workerlevel_plot)
println(fired_plot)
println(hired_plot)
println(filter_worker_df)

[119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
23×5 DataFrame
 Row │ Worker_Type  Date        Workers_Fired  Workers_Hired  Worker_Level 
     │ Any          String      Int64          Int64          Int64        
─────┼─────────────────────────────────────────────────────────────────────
   1 │ Tester       2024-02-01              0              0           119
   2 │ Tester       2024-03-01              0              0           119
   3 │ Tester       2024-04-01              0              0           119
   4 │ Tester       2024-05-01              0              0           119
   5 │ Tester       2024-06-01              0              0           119
   6 │ Tester       2024-07-01              0              0           119
   7 │ Tester       2024-08-01              0      